<a href="https://colab.research.google.com/github/Newtica/DataSets/blob/master/CIFAR_x_MNIST__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 110 kB/s 


In [ ]:
!pip3 install pytorch-metric-learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 15.7 MB/s 


In [ ]:
!pip install catalyst

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 14.4 MB/s 
     |████████████████████████████████| 175 kB 62.3 MB/s 
     |████████████████████████████████| 125 kB 68.1 MB/s 


In [ ]:
import torch
import torchvision
from torch import nn, optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import math
import torch.optim as optim

from torch.utils.data import DataLoader

from catalyst import dl, utils
from catalyst.contrib.datasets import MNIST
from catalyst.contrib.layers.arcface import ArcFace

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

from torchvision import datasets, transforms

print(torch.__version__)

1.12.1+cu113


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
#@title train-test split 4 CIFAR

from sklearn.model_selection import train_test_split

def train_test_interleave_split(classes, test_size):
    """Split classes into train and test subsets.
    Args:
        test_size: Fraction of the test in the [0, 1] range.
    Returns:
        Train classes and test classes.
    """
    classes1 = []
    classes2 = []
    s = 0
    for c in classes:
        s += 1 - test_size
        if s + 1e-6 > 1:
            s -= 1
            classes1.append(c)
        else:
            classes2.append(c)
    if not classes1 or not classes2:
        raise ValueError("Can't split into two non-empty datasets with the given fraction.")
    return classes1, classes2


class ClassSubsetDataset(torch.utils.data.Dataset):
    """Helper class for labels subset selection."""

    def __init__(self, dataset, classes):
        super().__init__()
        self._dataset = dataset
        self._indices = []
        labels = []
        classes = set(classes)
        for i in range(len(dataset)):
            label = dataset[i][1]
            if label not in classes:
                continue
            self._indices.append(i)
            labels.append(label)

        label_mapping = {label: i for i, label in enumerate(sorted(classes))}
        self._labels = [label_mapping[label] for label in labels]

    def __len__(self):
        return len(self._indices)

    def __getitem__(self, index):
        """Get element of the dataset.
        Classification dataset returns tuple (image, label).
        Verification dataset returns ((image1, image2), label).
        Datasets with quality assigned to each sample return tuples like
        (image, label, quality) or ((image1, image2), label, (quality1, quality2)).
        """
        return self._dataset[self._indices[index]]


class MergedDataset(torch.utils.data.Dataset):
    """Merge multiple datasets sharing the same set of labels."""
    def __init__(self, *datasets):
        super().__init__()
        if len(datasets) == 0:
            raise ValueError("Empty datasets list.")
        self._datasets = datasets

    def __len__(self):
        return sum(map(len, self._datasets))

    def __getitem__(self, index):
        """Get element of the dataset.
        Classification dataset returns tuple (image, label).
        Verification dataset returns ((image1, image2), label).
        Datasets with quality assigned to each sample return tuples like
        (image, label, quality) or ((image1, image2), label, (quality1, quality2)).
        """
        for dataset in self._datasets:
            if index < len(dataset):
                return dataset[index]
            index -= len(dataset)
        raise IndexError(index)


def split_classes(dataset, fraction=0.5, interleave=False):
    """Split dataset into two parts with different sets of labels.
    Function is deterministic. Split is based on hash values, not random.
    Returns:
        Two datasets. The size of the first dataset is proportional to fraction,
        the size of the second is proportional to (1 - fraction).
    """
    classes = list(sorted({dataset[i][1] for i in range(len(dataset))}))
    if interleave:
        classes1, classes2 = train_test_interleave_split(classes, test_size=1 - fraction)
    else:
        classes1, classes2 = train_test_split(classes, test_size=1 - fraction, shuffle=False)
    if not classes1 or not classes2:
        raise ValueError("Can't split into two non-empty datasets with the given fraction.")
    return ClassSubsetDataset(dataset, classes1), ClassSubsetDataset(dataset, classes2)

In [ ]:
#@title ArcFace as is
#https://catalyst-team.github.io/catalyst/api/contrib.html#arcface
#catalyst.contrib.layers.arcface.ArcFace(in_features: int, out_features: int, s: float = 64.0, m: float = 0.5, eps: float = 1e-06)
# Parameters
# in_features – size of each input sample
# out_features – size of each output sample.
# s – norm of input feature. Default: 64.0
# m – margin. Default: 0.5
# eps – operation accuracy. Default: 1e-6

# layer = ArcFace(5, 10, s=1.31, m=0.5)
# loss_fn = nn.CrossEntropyLoss()
# embedding = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(10)
# loss = loss_fn(output, target)
# self.engine.backward(loss)

In [ ]:
#@title ...
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

### 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 3)
        self.conv2 = nn.Conv2d(16, 16, 3, 3)
        #self.dropout1 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(144, 5)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        #x = F.max_pool2d(x, 2)
        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        #
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        #
        loss = loss_func(embeddings, labels)#, indices_tuple)
        ###losses.<...>(num_classes, embedding_size, margin=28.6, scale=64, **kwargs)

        
        #loss.backward()
        #
        optimizer.step()
        if batch_idx % 20 == 0:
            print(
                "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
                    epoch, batch_idx, loss, mining_func.num_triplets
                )
            )


# convenient function from pytorch-metric-learning
#
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)


# compute accuracy using AccuracyCalculator from pytorch-metric-learning
#
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing recall")
    accuracies = accuracy_calculator.get_accuracy(test_embeddings, test_embeddings, test_labels, test_labels, True)
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

#

In [ ]:
#@title СIFAR load

batch_size = 32

CIFAR_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
CIFAR_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
#CIFAR_classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
CIFAR_merged = MergedDataset(CIFAR_trainset, CIFAR_testset)
print(len(CIFAR_trainset), len(CIFAR_testset), len(CIFAR_merged))

CIFAR_trainset, CIFAR_testset = split_classes(CIFAR_merged)
print("Train labels", {label for _, label in CIFAR_trainset})
print("Test labels", {label for _, label in CIFAR_testset})
CIFAR_trainloader = torch.utils.data.DataLoader(CIFAR_trainset, batch_size=batch_size, shuffle=True, num_workers=2)
CIFAR_testloader = torch.utils.data.DataLoader(CIFAR_testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified
50000 10000 60000
Train labels {0, 1, 2, 3, 4}
Test labels {5, 6, 7, 8, 9}


In [ ]:
device = "cuda"
print(device)
model = Net().to(device)
#
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay = 0.0005)
num_epochs = 3

distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0)
layer = ArcFace(5, 10, s=1.31, m=0.5)
loss_func = nn.CrossEntropyLoss()
loss_func.num_triplets = 0
embedding = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(10)
# self.engine.backward(loss)  

accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)

for epoch in range(1, num_epochs + 1):
    #train(model, layer, loss_func, device, CIFAR_trainloader, optimizer, epoch)
    test(CIFAR_trainset, CIFAR_testset, model, accuracy_calculator)  

cuda


100%|██████████| 938/938 [00:08<00:00, 108.72it/s]


Computing recall
Test set accuracy (Precision@1) = 0.24313333333333334


 68%|██████▊   | 639/938 [00:06<00:02, 106.46it/s]


KeyboardInterrupt: ignored

In [ ]:
#@title train-test split 4 MNIST

import numpy as np
from sklearn.model_selection import train_test_split

def train_test_interleave_split(classes, test_size):
    """Split classes into train and test subsets.
    Args:
        test_size: Fraction of the test in the [0, 1] range.
    Returns:
        Train classes and test classes.
    """
    classes1 = []
    classes2 = []
    s = 0
    for c in classes:
        s += 1 - test_size
        if s + 1e-6 > 1:
            s -= 1
            classes1.append(c)
        else:
            classes2.append(c)
    if not classes1 or not classes2:
        raise ValueError("Can't split into two non-empty datasets with the given fraction.")
    return classes1, classes2


class ClassSubsetDataset(torch.utils.data.Dataset):
    """Helper class for labels subset selection."""

    def __init__(self, dataset, classes):
        super().__init__()
        self._dataset = dataset
        self._indices = []
        labels = []
        classes = set(classes)
        for i in range(len(dataset)):
            label = dataset[i][1]
            if label not in classes:
                continue
            self._indices.append(i)
            labels.append(label)

        label_mapping = {label: i for i, label in enumerate(sorted(classes))}
        self._labels = [label_mapping[label] for label in labels]

    def __len__(self):
        return len(self._indices)

    def __getitem__(self, index):
        """Get element of the dataset.
        Classification dataset returns tuple (image, label).
        Verification dataset returns ((image1, image2), label).
        Datasets with quality assigned to each sample return tuples like
        (image, label, quality) or ((image1, image2), label, (quality1, quality2)).
        """
        return self._dataset[self._indices[index]]


class MergedDataset(torch.utils.data.Dataset):
    """Merge multiple datasets sharing the same set of labels."""
    def __init__(self, *datasets):
        super().__init__()
        if len(datasets) == 0:
            raise ValueError("Empty datasets list.")
        self._datasets = datasets

    def __len__(self):
        return sum(map(len, self._datasets))

    def __getitem__(self, index):
        """Get element of the dataset.
        Classification dataset returns tuple (image, label).
        Verification dataset returns ((image1, image2), label).
        Datasets with quality assigned to each sample return tuples like
        (image, label, quality) or ((image1, image2), label, (quality1, quality2)).
        """
        for dataset in self._datasets:
            if index < len(dataset):
                return dataset[index]
            index -= len(dataset)
        raise IndexError(index)


def split_classes(dataset, fraction=0.5, interleave=False):
    """Split dataset into two parts with different sets of labels.
    Function is deterministic. Split is based on hash values, not random.
    Returns:
        Two datasets. The size of the first dataset is proportional to fraction,
        the size of the second is proportional to (1 - fraction).
    """
    classes = list(sorted({dataset[i][1] for i in range(len(dataset))}))
    if interleave:
        classes1, classes2 = train_test_interleave_split(classes, test_size=1 - fraction)
    else:
        classes1, classes2 = train_test_split(classes, test_size=1 - fraction, shuffle=False)
    if not classes1 or not classes2:
        raise ValueError("Can't split into two non-empty datasets with the given fraction.")
    return ClassSubsetDataset(dataset, classes1), ClassSubsetDataset(dataset, classes2)

In [ ]:
#@title MNIST


###
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        #self.dropout1 = nn.Dropout2d(0.25)
        #self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        #x = F.max_pool2d(x, 2)
        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        #
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        #
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        #
        optimizer.step()
        if batch_idx % 20 == 0:
            print(
                "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
                    epoch, batch_idx, loss, mining_func.num_triplets
                )
            )


# convenient function from pytorch-metric-learning
#
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)


# compute accuracy using AccuracyCalculator from pytorch-metric-learning
#
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing recall")
    accuracies = accuracy_calculator.get_accuracy(test_embeddings, test_embeddings, test_labels, test_labels, True)
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

#transform = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop(10), transforms.Normalize((0.1307,), (0.3081,)),])
#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#scripted_transforms = torch.jit.script(transforms)
#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

In [ ]:
#
batch_size = 32

dataset1 = datasets.MNIST(".", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST(".", train=False, transform=transform)

MNIST_merged = MergedDataset(dataset1, dataset2)
print(len(dataset1), len(dataset2), len(MNIST_merged))

dataset1, dataset2 = split_classes(MNIST_merged)
print("Train labels", {label for _, label in dataset1})
print("Test labels", {label for _, label in dataset2})
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size)

60000 10000 70000


RuntimeError: ignored

In [ ]:
#@title MNIST model
model = Net().to(device)
#
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay = 0.0005)
num_epochs = 1

# pytorch-metric-learning
distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0)
#loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)

layer = ArcFace(5, 10, s=1.31, m=0.5)
loss_func = nn.CrossEntropyLoss()
embedding = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(10)
# loss = loss_func(output, target)
#self.engine.backward(loss)

# For each element in the batch, this miner will find the hardest positive and hardest negative, and use those to form a single triplet. So for a batch size of N, this miner will output N triplets
#mining_func = miners.TripletMarginMiner(margin=0.2, distance=distance, type_of_triplets="semihard")

accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)

for epoch in range(1, num_epochs + 1):
    train(model, layer, loss_func, device, train_loader, optimizer, epoch)
    test(dataset1, dataset2, model, accuracy_calculator)

NameError: ignored